In [1]:

# Auto-reload pour développement interactif
%load_ext autoreload
%autoreload 2

# Imports standards
import pandas as pd
import numpy as np
import json
import spacy
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import hashlib
import re
from collections import Counter
import warnings
import sys

warnings.filterwarnings('ignore')

# NLP avancé
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

# Détection de langue
from langdetect import detect, detect_langs
from langdetect.lang_detect_exception import LangDetectException

# Preprocessing texte
import unicodedata
import ftfy  # Pour corriger les encodages
from bs4 import BeautifulSoup

# Similarité et déduplication
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import faiss






In [2]:

# Configuration
BASE_DIR = Path().resolve().parent.parent
sys.path.append(str(BASE_DIR / "src"))

# Répertoires
DATA_DIR = BASE_DIR / "data"
EXPORTS_DIR = DATA_DIR / "exports"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(exist_ok=True)

# Configuration des modèles
NLP_MODEL = "fr_core_news_sm"  # Modèle spaCy français LÉGER
EMBEDDINGS_MODEL = "all-MiniLM-L6-v2"  # Modèle plus léger (~80MB vs 420MB)
SIMILARITY_THRESHOLD = 0.85  # Seuil de similarité pour déduplication

print(f"Configuration optimisée pour ressources limitées:")
print(f"   SpaCy: {NLP_MODEL} (modèle léger)")
print(f"   Embeddings: {EMBEDDINGS_MODEL} (80MB au lieu de 420MB)")



Configuration optimisée pour ressources limitées:
   SpaCy: fr_core_news_sm (modèle léger)
   Embeddings: all-MiniLM-L6-v2 (80MB au lieu de 420MB)


In [3]:
# Chargement du fichier JSON enrichi (priorité) ou brut (fallback)
enriched_file = EXPORTS_DIR / "enriched_article.json"  # MODIFIÉ: sans "s"
enriched_files_alt = EXPORTS_DIR / "enriched_articles.json"  # Alternative
raw_file = EXPORTS_DIR / "raw_articles.json"



In [4]:
# Détection automatique du fichier source
if enriched_file.exists():
    source_file = enriched_file
    print(f"FICHIER ENRICHI DÉTECTÉ: {enriched_file}")
    print("   Mode: Preprocessing avancé sur données pré-enrichies")
elif enriched_files_alt.exists():
    source_file = enriched_files_alt
    print(f"FICHIER ENRICHI ALTERNATIF DÉTECTÉ: {enriched_files_alt}")
    print("   Mode: Preprocessing avancé sur données pré-enrichies")
elif raw_file.exists():
    source_file = raw_file
    print(f"FICHIER BRUT DÉTECTÉ: {raw_file}")
    print("   → Mode: Preprocessing complet depuis zéro")
else:
    print(f"ERREUR: Aucun fichier source trouvé!")
    print(f"   Recherche: {enriched_file} OU {raw_file}")
    print("   Solution: Exécutez d'abord collect_articles.ipynb ou enrich_articles.ipynb")
    exit(1)

with open(source_file, 'r', encoding='utf-8') as f:
    articles_data = json.load(f)



FICHIER ENRICHI ALTERNATIF DÉTECTÉ: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\exports\enriched_articles.json
   Mode: Preprocessing avancé sur données pré-enrichies


In [5]:
# Mode adaptatif selon la source
ENRICHED_MODE = "enriched_article" in str(source_file)  # Compatible avec les deux formats

print(f"{len(articles_data)} articles chargés depuis {source_file.name}")



200 articles chargés depuis enriched_articles.json


In [6]:
# Conversion en DataFrame pour manipulation
df = pd.DataFrame(articles_data)



In [7]:
df

,id,title,summary,text,published,source,url,created_at,cleaned_text,language,entities,embedding,quality_score
0,26,Top 10 AI Content Generator & Writer Tools in ...,Are you looking for a way to create content th...,Top 10 AI Content Generator & Writer Tools in ...,2022-11-15T08:58:00,https://feeds.feedburner.com/TechCrunch/,https://techncruncher.blogspot.com/2022/11/top...,2025-07-26T05:41:35.784294,Top 10 AI Content Generator & Writer Tools in ...,en,{'MISC': ['10 writing tools that used AI to cr...,"[-0.11071627587080002, -0.041722897440195084, ...",0.142
1,569,How I Fine-Tuned Granite-Vision 2B to Beat a 9...,A hands-on journey exploring fine-tuning techn...,or vision-language models is a powerful techni...,2025-07-25T19:29:01,https://towardsdatascience.com/feed,https://towardsdatascience.com/how-i-fine-tune...,2025-07-26T06:23:31.282901,or vision-language models is a powerful techni...,en,"{'PER': ['Ada Lovelace', 'Despite', 'Eli Schwa...","[-0.06249945983290672, -0.03206057474017143, -...",0.146
2,586,When LLMs Try to Reason: Experiments in Text a...,Can large language models learn to reason abst...,models learn to reason abstractly from just a ...,2025-07-22T19:35:25,https://towardsdatascience.com/feed,https://towardsdatascience.com/when-llms-try-t...,2025-07-26T06:23:31.390578,models learn to reason abstractly from just a ...,en,"{'MISC': ['# Black '#', '# Here is an extracte...","[0.004933114629238844, -0.06849845498800278, 0...",0.153
3,263,"""On n'a fait que du Tetris"" : on vous raconte ...",Vous pensiez tout savoir du spectacle mis en s...,Vous pensiez tout savoir du spectacle mis en s...,2025-07-26T05:00:37,https://www.francetvinfo.fr/titres.rss,https://www.franceinfo.fr/les-jeux-olympiques/...,2025-07-26T05:53:42.336454,Vous pensiez tout savoir du spectacle mis en s...,fr,"{'PER': ['Alexandre Boyon', 'Alexis Decalonne'...","[-0.0072211455553770065, 0.07925660908222198, ...",0.179
4,578,How Not to Mislead with Your Data-Driven Story,Data storytelling can enlighten—but it can als...,"is everywhere. There are countless books, arti...",2025-07-23T19:10:41,https://towardsdatascience.com/feed,https://towardsdatascience.com/how-not-to-misl...,2025-07-26T06:23:31.338091,"is everywhere. There are countless books, arti...",en,"{'MISC': ['A clean', 'AI chatbots distort and ...","[0.05727158114314079, 0.09395787864923477, 0.0...",0.194
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,14,​​What is inside the GHF food aid box being di...,BBC Verify examines the meals handed out to Ga...,​​What is inside the GHF food aid box being di...,2025-07-25T23:41:52,https://feeds.bbci.co.uk/news/world/rss.xml,https://www.bbc.com/news/articles/cd787er1qz4o,2025-07-26T05:41:35.648879,​​What is inside the GHF food aid box being di...,en,"{'MISC': ['Dr Andrew Seal', 'GHF', 'He added t...","[-0.012845661491155624, 0.027754053473472595, ...",0.243
196,527,Alibaba’s new Qwen reasoning AI model sets ope...,The Qwen team from Alibaba have just released ...,The Qwen team from Alibaba have just released ...,2025-07-25T12:43:07,https://www.artificialintelligence-news.com/feed/,https://www.artificialintelligence-news.com/ne...,2025-07-26T05:53:46.540024,The Qwen team from Alibaba have just released ...,en,"{'MISC': ['AI model from the Qwen team with', ...","[-0.06894423812627792, -0.08636637032032013, -...",0.243
197,472,Plus de 20 000 suppressions de postes chez Int...,Intel taille dans le vif. Moins de cinq mois a...,Intel taille dans le vif. Moins de cinq mois a...,2025-07-25T12:46:24,https://www.01net.com/rss/actualites/,https://www.01net.com/actualites/plus-de-20-00...,2025-07-26T05:53:45.935827,Intel taille dans le vif. Moins de cinq mois a...,fr,"{'ORG': ['CNBC', 'Intel', 'Lip-Bu Tan', 'Micro...","[-0.02674338035285473, 0.021592628210783005, 0...",0.256
198,309,Centrafrique: la pression se renforce sur le g...,"En Centrafrique, la pression s'accentue sur le...","En Centrafrique, la pression s'accentue sur le...",2025-07-25T23:05:15,

In [7]:
# Harmonisation text_cleaned vs cleaned_text
if 'cleaned_text' in df.columns and 'text_cleaned' not in df.columns:
    df['text_cleaned'] = df['cleaned_text']
    print("   Harmonisation: cleaned_text → text_cleaned")
elif 'text' in df.columns and 'text_cleaned' not in df.columns:
    df['text_cleaned'] = df['text']
    print("   Création: text → text_cleaned")

# Vérification des colonnes essentielles
required_columns = ['title', 'text', 'source']
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"   Colonnes manquantes: {missing_columns}")
else:
    print("   Toutes les colonnes essentielles présentes")

print(f"   Structure: {len(df)} articles, {len(df.columns)} colonnes")



   Harmonisation: cleaned_text → text_cleaned
   Toutes les colonnes essentielles présentes
   Structure: 200 articles, 14 colonnes


In [8]:
# Détection des colonnes d'enrichissement déjà présentes
enrichment_columns = {
    'language': 'language' in df.columns and df['language'].notna().sum() > 0,
    'entities': 'entities' in df.columns and df['entities'].notna().sum() > 0,
    'quality_score': 'quality_score' in df.columns and df['quality_score'].notna().sum() > 0,
    'embedding': 'embedding' in df.columns and df['embedding'].notna().sum() > 0
}


In [9]:
for col, present in enrichment_columns.items():
    status = "Présent" if present else "Absent"
    count = df[col].notna().sum() if present else 0
    print(f"   {col}: {status} ({count} articles)")

# Adaptation de la stratégie
if enrichment_columns['language'] and enrichment_columns['entities']:
    print(f"\nMODE DÉTECTÉ: Preprocessing complémentaire avancé")
    print(f"   → Focus: Déduplication, biais, corpus calibration, métriques avancées")
    SKIP_BASIC_ENRICHMENT = True
else:
    print(f"\nMODE DÉTECTÉ: Preprocessing complet depuis zéro")
    print(f"   → Pipeline: Enrichissement + Analyses avancées")
    SKIP_BASIC_ENRICHMENT = False



   language: Présent (200 articles)
   entities: Présent (200 articles)
   quality_score: Présent (200 articles)
   embedding: Présent (200 articles)

MODE DÉTECTÉ: Preprocessing complémentaire avancé
   → Focus: Déduplication, biais, corpus calibration, métriques avancées


In [10]:
def clean_text_advanced(text):
    """Nettoyage robuste et avancé du texte"""
    if pd.isna(text) or not text:
        return ""
    
    # Correction encodage
    text = ftfy.fix_text(text)
    # Suppression HTML résiduel
    text = BeautifulSoup(text, "html.parser").get_text()
    # Normalisation Unicode
    text = unicodedata.normalize('NFKC', text)
    # Suppression caractères de contrôle
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    # Normalisation espaces
    text = re.sub(r'\s+', ' ', text)
    # Suppression URLs et emails
    text = re.sub(r'http[s]?://\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    # Suppression patterns RSS spécifiques
    text = re.sub(r'#xtor=RSS-\d+.*', '', text)
    text = re.sub(r'\[.*?\]$', '', text)  # Crédits en fin d'article
    
    return text.strip()

# Application du nettoyage si nécessaire
if 'text_cleaned' not in df.columns or df['text_cleaned'].isna().any():
    print("   Application du nettoyage avancé...")
    df['text_cleaned'] = df['text'].apply(clean_text_advanced)
    print(f"      {len(df)} textes nettoyés")
else:
    print("   Textes déjà nettoyés détectés")

# Filtrage des articles trop courts ou vides
min_length = 100  # caractères minimum
df_clean = df[df['text_cleaned'].str.len() >= min_length].copy()
print(f"   Filtrage longueur minimum ({min_length} chars): {len(df)} → {len(df_clean)} articles")




   Textes déjà nettoyés détectés
   Filtrage longueur minimum (100 chars): 200 → 200 articles


In [11]:
if not enrichment_columns['language'] or not SKIP_BASIC_ENRICHMENT:
    print("   Détection de langue en cours...")
    def detect_language_robust(text):
        """Détection de langue avec fallback"""
        if not text or len(text) < 50:
            return 'unknown', 0.0
        
        try:
            # langdetect avec probabilités
            langs = detect_langs(text)
            primary_lang = langs[0]
            return primary_lang.lang, primary_lang.prob
        except LangDetectException:
            # Fallback: détection basique
            try:
                return detect(text), 0.5
            except:
                return 'unknown', 0.0

    # Application de la détection
    language_results = df_clean['text_cleaned'].apply(detect_language_robust)
    df_clean['language'] = [result[0] for result in language_results]
    df_clean['language_confidence'] = [result[1] for result in language_results]
    
    print(f"   Détection de langue terminée")
else:
    print("   Langues déjà détectées, validation des données...")
    if 'language_confidence' not in df_clean.columns:
        df_clean['language_confidence'] = df_clean['language'].apply(lambda x: 0.9 if x == 'fr' else 0.7)

# Sélection intelligente selon la distribution
lang_counts = df_clean['language'].value_counts()
if lang_counts.get('fr', 0) > len(df_clean) * 0.3:  # Si >30% en français
    df_filtered = df_clean[df_clean['language'] == 'fr'].copy()
    print(f"   Focus français: {len(df_filtered)} articles sélectionnés")
else:
    # Garder top 2 langues si pas assez de français
    top_langs = lang_counts.head(2).index.tolist()
    df_filtered = df_clean[df_clean['language'].isin(top_langs)].copy()
    print(f"   Multi-langues: {len(df_filtered)} articles ({top_langs})")



   Langues déjà détectées, validation des données...
   Focus français: 130 articles sélectionnés


In [12]:
# Chargement du modèle d'embeddings avec gestion d'erreur
print("   Chargement du modèle sentence-transformers...")
try:
    embeddings_model = SentenceTransformer(EMBEDDINGS_MODEL)
    print(f"      Modèle {EMBEDDINGS_MODEL} chargé avec succès")
except OSError as e:
    if "pagination" in str(e) or "1455" in str(e):
        print(f"      Erreur mémoire détectée, utilisation d'un modèle plus léger...")
        # Fallback vers un modèle encore plus petit
        EMBEDDINGS_MODEL_FALLBACK = "all-MiniLM-L6-v2"
        embeddings_model = SentenceTransformer(EMBEDDINGS_MODEL_FALLBACK)
        print(f"      Modèle fallback {EMBEDDINGS_MODEL_FALLBACK} chargé")
    else:
        raise e

def deduplicate_semantic(df, threshold=0.85):
    """Déduplication sémantique avancée avec FAISS et correction du type"""
    print(f"   Génération des embeddings pour {len(df)} articles...")
    
    # Utilisation des embeddings existants ou génération
    if 'embedding' in df.columns and df['embedding'].notna().sum() > 0:
        print("      Utilisation des embeddings existants")
        embeddings = []
        for idx, emb in df['embedding'].items():
            if isinstance(emb, (list, np.ndarray)) and len(emb) > 0:
                embeddings.append(np.array(emb, dtype=np.float32))  # CORRECTION: forcer float32
            else:
                # Génération pour les embeddings manquants
                text = df.loc[idx, 'text_cleaned']
                new_emb = embeddings_model.encode(text)
                embeddings.append(np.array(new_emb, dtype=np.float32))  # CORRECTION: forcer float32
        embeddings = np.array(embeddings, dtype=np.float32)  # CORRECTION: forcer float32
    else:
        print("      Génération des embeddings...")
        texts = df['text_cleaned'].tolist()
        embeddings = embeddings_model.encode(texts, show_progress_bar=True)
        embeddings = np.array(embeddings, dtype=np.float32)  # CORRECTION: forcer float32

    # Configuration FAISS
    print("   Configuration de l'index FAISS...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)

    # CORRECTION: Normalisation avec vérification du type et copie
    embeddings_normalized = embeddings.copy()  # Créer une copie
    if not embeddings_normalized.flags['C_CONTIGUOUS']:
        embeddings_normalized = np.ascontiguousarray(embeddings_normalized)  # Assurer la contiguïté

    faiss.normalize_L2(embeddings_normalized)  # Normaliser la copie
    index.add(embeddings_normalized)
    
    # Recherche des doublons
    print("   Recherche des doublons sémantiques...")
    similarities, indices = index.search(embeddings_normalized, k=5)
    
    to_remove = set()
    duplicate_pairs = []
    
    for i, (sim_scores, sim_indices) in enumerate(zip(similarities, indices)):
        for j, (score, idx) in enumerate(zip(sim_scores, sim_indices)):
            if j > 0 and score > threshold and idx not in to_remove and i not in to_remove:
                # Garde le plus récent ou le mieux noté
                if df.iloc[i].get('quality_score', 0) >= df.iloc[idx].get('quality_score', 0):
                    to_remove.add(idx)
                else:
                    to_remove.add(i)
                
                duplicate_pairs.append((i, idx, score))
    
    # Suppression des doublons
    df_dedup = df.drop(df.index[list(to_remove)]).copy()
    
    print(f"   Résultats déduplication:")
    print(f"      Articles originaux: {len(df)}")
    print(f"      Doublons détectés: {len(to_remove)}")
    print(f"      Articles finaux: {len(df_dedup)}")
    print(f"      Taux de déduplication: {len(to_remove)/len(df)*100:.1f}%")

    return df_dedup, embeddings_normalized



   Chargement du modèle sentence-transformers...
      Modèle all-MiniLM-L6-v2 chargé avec succès


In [13]:
# Application de la déduplication
df_clean_dedup, article_embeddings = deduplicate_semantic(df_filtered, SIMILARITY_THRESHOLD)


   Génération des embeddings pour 130 articles...
      Utilisation des embeddings existants
   Configuration de l'index FAISS...
   Recherche des doublons sémantiques...
   Résultats déduplication:
      Articles originaux: 130
      Doublons détectés: 2
      Articles finaux: 128
      Taux de déduplication: 1.5%


In [14]:

if not enrichment_columns['entities'] or not SKIP_BASIC_ENRICHMENT:
    print("   Extraction complète des entités avec spaCy...")
    
    # Chargement du modèle spaCy français
    print(f"      Chargement du modèle spaCy: {NLP_MODEL}")
    try:
        nlp = spacy.load(NLP_MODEL)
    except OSError:
        print(f"      Modèle {NLP_MODEL} non trouvé. Installation...")
        import subprocess
        subprocess.run(f"python -m spacy download {NLP_MODEL}", shell=True)
        nlp = spacy.load(NLP_MODEL)

    def extract_entities_advanced(text, nlp_model):
        """Extraction d'entités avec enrichissements"""
        if not text or len(text) < 50:
            return {
                'persons': [], 'organizations': [], 'locations': [],
                'dates': [], 'money': [], 'misc': []
            }
        
        # Traitement avec spaCy (limiter la longueur pour performance)
        doc = nlp_model(text[:8000])  # Premier 8k caractères
        
        entities = {
            'persons': [],
            'organizations': [],
            'locations': [],
            'dates': [],
            'money': [],
            'misc': []
        }
        
        for ent in doc.ents:
            entity_text = ent.text.strip()
            if len(entity_text) < 2:  # Ignorer entités trop courtes
                continue
                
            if ent.label_ in ['PERSON']:
                entities['persons'].append(entity_text)
            elif ent.label_ in ['ORG']:
                entities['organizations'].append(entity_text)
            elif ent.label_ in ['GPE', 'LOC']:
                entities['locations'].append(entity_text)
            elif ent.label_ in ['DATE', 'TIME']:
                entities['dates'].append(entity_text)
            elif ent.label_ in ['MONEY']:
                entities['money'].append(entity_text)
            else:
                entities['misc'].append(entity_text)
        
        # Déduplication et nettoyage
        for key in entities:
            entities[key] = list(set(entities[key]))  # Suppression doublons
            entities[key] = [e for e in entities[key] if len(e) > 1]  # Filtrage longueur
        
        return entities

    # Application de l'extraction d'entités
    entities_results = []
    for text in tqdm(df_clean_dedup['text_cleaned'], desc="Extraction NER"):
        entities = extract_entities_advanced(text, nlp)
        entities_results.append(entities)

    # Ajout des résultats au DataFrame
    df_clean_dedup['entities_advanced'] = entities_results
    
    print(f"      Extraction NER terminée")
else:
    print("   Conversion et amélioration des entités existantes...")
    def convert_and_improve_entities(existing_entities):
        """CONVERSION du format existant + amélioration"""
        if not existing_entities or not isinstance(existing_entities, dict):
            return {
                'persons': [], 'organizations': [], 'locations': [],
                'dates': [], 'money': [], 'misc': []
            }
        
        # CONVERSION DES FORMATS
        converted = {
            'persons': [],
            'organizations': [],
            'locations': [],
            'dates': [],
            'money': [],
            'misc': []
        }
        
        # Mapping des anciens noms vers les nouveaux
        field_mapping = {
            'PER': 'persons',           # PER → persons
            'PERSON': 'persons',        # PERSON → persons  
            'ORG': 'organizations',     # ORG → organizations
            'LOC': 'locations',         # LOC → locations
            'GPE': 'locations',         # GPE → locations (entités géopolitiques)
            'MISC': 'misc',             # MISC → misc
            'DATE': 'dates',            # DATE → dates
            'TIME': 'dates',            # TIME → dates
            'MONEY': 'money'            # MONEY → money
        }
        
        # Conversion avec mapping
        for old_key, entity_list in existing_entities.items():
            if old_key in field_mapping:
                new_key = field_mapping[old_key]
                if isinstance(entity_list, list):
                    # Nettoyage des entités
                    cleaned = [str(e).strip() for e in entity_list if e and len(str(e)) > 1]
                    # Déduplication case-insensitive
                    seen = set()
                    for entity in cleaned:
                        if entity.lower() not in seen:
                            converted[new_key].append(entity)
                            seen.add(entity.lower())
        
        return converted

    # Application de la conversion
    df_clean_dedup['entities_advanced'] = df_clean_dedup['entities'].apply(convert_and_improve_entities)


   Conversion et amélioration des entités existantes...


In [15]:
# Création de colonnes métriques enrichies
df_clean_dedup['persons_count'] = df_clean_dedup['entities_advanced'].apply(lambda x: len(x.get('persons', [])))
df_clean_dedup['organizations_count'] = df_clean_dedup['entities_advanced'].apply(lambda x: len(x.get('organizations', [])))
df_clean_dedup['locations_count'] = df_clean_dedup['entities_advanced'].apply(lambda x: len(x.get('locations', [])))
df_clean_dedup['entities_total'] = (df_clean_dedup['persons_count'] + 
                                   df_clean_dedup['organizations_count'] +
                                   df_clean_dedup['locations_count'])

print(f"   Résumé entités: {df_clean_dedup['entities_total'].mean():.1f} entités/article en moyenne")



   Résumé entités: 23.1 entités/article en moyenne


In [16]:
# Téléchargement des ressources NLTK si nécessaire
try:
    import ssl
    _create_unverified_https_context = ssl._create_unverified_context
    ssl._create_default_https_context = _create_unverified_https_context
except:
    pass
nltk.download('punkt', quiet=True)

def segment_text_advanced(text):
    """Segmentation en phrases avec analyse sémantique avancée"""
    if not text or len(text) < 100:
        return {
            'sentences': [],
            'sentence_count': 0,
            'avg_sentence_length': 0,
            'paragraphs': [],
            'paragraph_count': 0,
            'text_complexity': 0,
            'readability_score': 0
        }
    
    # Segmentation en phrases (multi-langue)
    sentences = sent_tokenize(text[:5000], language='french')  # Limiter pour performance
    
    # Segmentation en paragraphes
    paragraphs = [p.strip() for p in text.split('\n\n') if p.strip() and len(p) > 20]

    # Métriques avancées
    sentence_lengths = [len(s.split()) for s in sentences]
    avg_sentence_length = np.mean(sentence_lengths) if sentence_lengths else 0
    
    # Score de complexité basé sur la longueur des phrases
    complexity = 0
    if sentence_lengths:
        variance = np.var(sentence_lengths)
        long_sentences = sum(1 for length in sentence_lengths if length > 20)
        complexity = min((variance / 100) + (long_sentences / len(sentences)), 1.0)

    # Score de lisibilité approximatif (Flesch-like)
    if sentence_lengths and avg_sentence_length > 0:
        readability = max(0, min(1, 1 - (avg_sentence_length - 10) / 20))
    else:
        readability = 0.5

    return {
        'sentences': sentences[:50],  # Limiter pour stockage
        'sentence_count': len(sentences),
        'avg_sentence_length': avg_sentence_length,
        'paragraphs': paragraphs[:20],  # Limiter pour stockage
        'paragraph_count': len(paragraphs),
        'text_complexity': complexity,
        'readability_score': readability
    }



In [17]:
# Application de la segmentation
print("   Segmentation  en cours...")
segmentation_results = []
for text in tqdm(df_clean_dedup['text_cleaned'], desc="Segmentation"):
    segments = segment_text_advanced(text)
    segmentation_results.append(segments)

df_clean_dedup['segmentation'] = segmentation_results


   Segmentation  en cours...


Segmentation: 100%|██████████| 128/128 [00:00<00:00, 801.69it/s]


In [18]:
# Extraction des métriques de segmentation
df_clean_dedup['sentence_count'] = df_clean_dedup['segmentation'].apply(lambda x: x['sentence_count'])
df_clean_dedup['paragraph_count'] = df_clean_dedup['segmentation'].apply(lambda x: x['paragraph_count'])
df_clean_dedup['avg_sentence_length'] = df_clean_dedup['segmentation'].apply(lambda x: x['avg_sentence_length'])
df_clean_dedup['text_complexity'] = df_clean_dedup['segmentation'].apply(lambda x: x['text_complexity'])
df_clean_dedup['readability_score'] = df_clean_dedup['segmentation'].apply(lambda x: x['readability_score'])

print(f"   Résumé segmentation: {df_clean_dedup['sentence_count'].mean():.1f} phrases/article")



   Résumé segmentation: 29.7 phrases/article


In [19]:
# Conversion et nettoyage des dates
print("   Analyse temporelle...")
df_clean_dedup['published_clean'] = pd.to_datetime(df_clean_dedup['published'], errors='coerce')

# Extraction des composants temporels
df_clean_dedup['hour'] = df_clean_dedup['published_clean'].dt.hour
df_clean_dedup['day_of_week'] = df_clean_dedup['published_clean'].dt.day_name()
df_clean_dedup['month'] = df_clean_dedup['published_clean'].dt.month
df_clean_dedup['date_only'] = df_clean_dedup['published_clean'].dt.date



   Analyse temporelle...


In [20]:
# Analyse des biais temporels
valid_dates = df_clean_dedup[df_clean_dedup['published_clean'].notna()]
if len(valid_dates) > 0:
    # Distribution horaire
    hour_dist = valid_dates['hour'].value_counts().head(3)
    # Distribution par jour
    day_dist = valid_dates['day_of_week'].value_counts().head(3)
    
    # Calcul du score de biais temporel
    hour_entropy = -sum((p := hour_dist / len(valid_dates)) * np.log2(p + 1e-10))
    day_entropy = -sum((p := day_dist / len(valid_dates)) * np.log2(p + 1e-10))
    
    # Normalisation (entropie max = log2(24) pour heures, log2(7) pour jours)
    hour_bias = 1 - (hour_entropy / np.log2(24))  # 0 = uniforme, 1 = très biaisé
    day_bias = 1 - (day_entropy / np.log2(7))
    
    print(f"      Biais temporel détecté: {hour_bias:.2f} (horaire), {day_bias:.2f} (quotidien)")



      Biais temporel détecté: 0.81 (horaire), 0.50 (quotidien)


In [21]:
# Analyse géographique via entités lieux
print("   Analyse géographique...")
all_locations = []
location_counts_by_article = []
for entities in df_clean_dedup['entities_advanced']:
    article_locations = entities.get('locations', [])
    location_counts_by_article.append(len(article_locations))
    all_locations.extend(article_locations)

location_distribution = Counter(all_locations)
df_clean_dedup['locations_count'] = location_counts_by_article

if location_distribution:
    # Score de biais géographique
    if len(location_distribution) > 1:
        geo_probs = np.array(list(location_distribution.values())) / len(all_locations)
        geo_entropy = -sum(geo_probs * np.log2(geo_probs + 1e-10))
        max_entropy = np.log2(min(len(location_distribution), 50))  # Entropie max théorique
        geo_bias = 1 - (geo_entropy / max_entropy) if max_entropy > 0 else 0
        print(f"      Biais géographique détecté: {geo_bias:.2f}")
    else:
        geo_bias = 1.0  # Maximum bias if only one location
        print(f"      Biais géographique maximal détecté")







   Analyse géographique...
      Biais géographique détecté: -0.57


In [22]:
# Ajout des scores de biais au DataFrame
df_clean_dedup['temporal_bias_hour'] = hour_bias if 'hour_bias' in locals() else 0
df_clean_dedup['temporal_bias_day'] = day_bias if 'day_bias' in locals() else 0
df_clean_dedup['geographic_bias'] = geo_bias if 'geo_bias' in locals() else 0


In [23]:
def calculate_advanced_quality_score(row):
    """Calcul d'un score de qualité multi-dimensionnel"""
    scores = {}
    
    # 1. Score de longueur (0-1)
    text_len = len(str(row.get('text_cleaned', '')))
    scores['length'] = min(text_len / 2000, 1.0)  # Optimal à 2000 caractères
    
    # 2. Score d'entités (0-1)
    entities_count = row.get('entities_total', 0)
    scores['entities'] = min(entities_count / 10, 1.0)  # Optimal à 10 entités
    
    # 3. Score de lisibilité (0-1)
    scores['readability'] = row.get('readability_score', 0.5)
    
    # 4. Score de complexité inversé (0-1)
    complexity = row.get('text_complexity', 0.5)
    scores['complexity'] = 1 - complexity  # Moins complexe = meilleur
    
    # 5. Score de structure (0-1)
    sentence_count = row.get('sentence_count', 0)
    paragraph_count = row.get('paragraph_count', 0)
    if sentence_count > 0 and paragraph_count > 0:
        structure_ratio = min(sentence_count / paragraph_count, 10) / 10  # Ratio phrases/paragraphes
        scores['structure'] = structure_ratio
    else:
        scores['structure'] = 0.1
        
    # 6. Score de langue (0-1)
    lang_confidence = row.get('language_confidence', 0.5)
    scores['language'] = lang_confidence
    
    # Score global pondéré
    weights = {
        'length': 0.2,
        'entities': 0.25, 
        'readability': 0.2,
        'complexity': 0.15,
        'structure': 0.1,
        'language': 0.1
    }
    
    final_score = sum(scores[key] * weights[key] for key in scores)
    
    return {
        'quality_score_advanced': final_score,
        'quality_breakdown': scores
    }

In [24]:
# Application du calcul de qualité
print("   Calcul des scores de qualité...")
quality_results = []
for _, row in tqdm(df_clean_dedup.iterrows(), total=len(df_clean_dedup), desc="Qualité"):
    quality_result = calculate_advanced_quality_score(row)
    quality_results.append(quality_result)

# Ajout des résultats
df_clean_dedup['quality_score_advanced'] = [r['quality_score_advanced'] for r in quality_results]
df_clean_dedup['quality_breakdown'] = [r['quality_breakdown'] for r in quality_results]

print(f"   Score qualité moyen: {df_clean_dedup['quality_score_advanced'].mean():.3f}")





   Calcul des scores de qualité...


Qualité: 100%|██████████| 128/128 [00:00<00:00, 7533.34it/s]

   Score qualité moyen: 0.681


In [25]:
def create_stratified_calibration_corpus(df, n_samples=300):
    """Création d'un corpus stratifié pour calibration"""
    
    # Définition des strates multi-dimensionnelles
    print("   Définition des strates...")
    
    # 1. Strate par qualité (3 niveaux)
    quality_tertiles = df['quality_score_advanced'].quantile([0.33, 0.67])
    df['quality_stratum'] = pd.cut(df['quality_score_advanced'], 
                                  bins=[0, quality_tertiles[0.33], quality_tertiles[0.67], 1],
                                  labels=['low', 'medium', 'high'])
    
    # 2. Strate par longueur (3 niveaux)
    df['text_length'] = df['text_cleaned'].str.len()
    length_tertiles = df['text_length'].quantile([0.33, 0.67])
    df['length_stratum'] = pd.cut(df['text_length'],
                                 bins=[0, length_tertiles[0.33], length_tertiles[0.67], float('inf')],
                                 labels=['short', 'medium', 'long'])
    
    # 3. Strate par richesse en entités (3 niveaux)
    if df['entities_total'].max() > 0:
        entity_tertiles = df['entities_total'].quantile([0.33, 0.67])
        df['entity_stratum'] = pd.cut(df['entities_total'],
                                     bins=[-1, entity_tertiles[0.33], entity_tertiles[0.67], float('inf')],
                                     labels=['sparse', 'moderate', 'rich'])
    else:
        df['entity_stratum'] = 'sparse'
        
    # 4. Strate par source (top sources + autres)
    source_counts = df['source'].value_counts()
    top_sources = source_counts.head(5).index.tolist()
    df['source_stratum'] = df['source'].apply(lambda x: x if x in top_sources else 'other')
    
    # Échantillonnage stratifié proportionnel
    print("   Échantillonnage stratifié...")
    
    # Groupement par strates multiples
    strata_cols = ['quality_stratum', 'length_stratum', 'entity_stratum', 'source_stratum']
    grouped = df.groupby(strata_cols, group_keys=False)
    
    # Calcul des tailles d'échantillon par strate
    strata_sizes = grouped.size()
    total_size = len(df)
    
    sample_dfs = []
    remaining_samples = n_samples
    
    for stratum, group in grouped:
        if remaining_samples <= 0:
            break
            
        # Taille proportionnelle de l'échantillon pour cette strate
        stratum_size = len(group)
        proportion = stratum_size / total_size
        target_sample_size = max(1, int(proportion * n_samples))
        
        # Ajustement si on dépasse le nombre d'échantillons restants
        actual_sample_size = min(target_sample_size, remaining_samples, stratum_size)
        
        if actual_sample_size > 0:
            # Échantillonnage au sein de la strate
            if len(group) >= actual_sample_size:
                # Tri par score de qualité pour prendre les meilleurs
                group_sorted = group.sort_values('quality_score_advanced', ascending=False)
                stratum_sample = group_sorted.head(actual_sample_size)
                sample_dfs.append(stratum_sample)
                remaining_samples -= actual_sample_size
                
    # Combinaison des échantillons de toutes les strates
    if sample_dfs:
        calibration_corpus = pd.concat(sample_dfs, ignore_index=True)
    else:
        # Fallback: échantillonnage simple par qualité
        calibration_corpus = df.nlargest(n_samples, 'quality_score_advanced')
        
    # Complément aléatoire si nécessaire
    if len(calibration_corpus) < n_samples:
        remaining_df = df[~df.index.isin(calibration_corpus.index)]
        if len(remaining_df) > 0:
            additional_samples = min(n_samples - len(calibration_corpus), len(remaining_df))
            additional = remaining_df.sample(n=additional_samples, random_state=42)
            calibration_corpus = pd.concat([calibration_corpus, additional], ignore_index=True)
            
    return calibration_corpus.head(n_samples)  # S'assurer qu'on ne dépasse pas



In [26]:
# Création du corpus de calibration
calibration_corpus = create_stratified_calibration_corpus(df_clean_dedup, n_samples=300)

print(f"   CORPUS DE CALIBRATION CRÉÉ:")
print(f"      Taille finale: {len(calibration_corpus)} articles")
print(f"      Score qualité moyen: {calibration_corpus['quality_score_advanced'].mean():.3f}")



   Définition des strates...
   Échantillonnage stratifié...
   CORPUS DE CALIBRATION CRÉÉ:
      Taille finale: 186 articles
      Score qualité moyen: 0.679


In [27]:
# Métriques de qualité finales
quality_metrics = {
    'source_file': str(source_file.name),
    'enriched_mode': ENRICHED_MODE,
    'total_articles_input': len(articles_data),
    'articles_after_deduplication': len(df_clean_dedup),
    'calibration_corpus_size': len(calibration_corpus),
    'deduplication_rate': ((len(df) - len(df_clean_dedup)) / len(df)) if len(df) > 0 else 0,
    'avg_quality_score': df_clean_dedup['quality_score_advanced'].mean(),
    'language_distribution': df_clean_dedup['language'].value_counts().to_dict(),
    'entities_avg_per_article': df_clean_dedup['entities_total'].mean(),
    'temporal_bias_detected': df_clean_dedup['temporal_bias_hour'].iloc[0] if len(df_clean_dedup) > 0 else 0,
    'geographic_bias_detected': df_clean_dedup['geographic_bias'].iloc[0] if len(df_clean_dedup) > 0 else 0,
    'processing_timestamp': datetime.now().isoformat()
}



In [28]:
# Sauvegarde du DataFrame principal (format optimisé)
output_file = PROCESSED_DIR / "articles_preprocessed_advanced.pkl"
df_clean_dedup.to_pickle(output_file)
print(f"   Dataset principal sauvegardé: {output_file}")



   Dataset principal sauvegardé: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\processed\articles_preprocessed_advanced.pkl


In [29]:
# Sauvegarde du corpus de calibration
calibration_file = PROCESSED_DIR / "calibration_corpus_stratified.pkl"
calibration_corpus.to_pickle(calibration_file)
print(f"   Corpus de calibration sauvegardé: {calibration_file}")



   Corpus de calibration sauvegardé: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\processed\calibration_corpus_stratified.pkl


In [30]:
# Sauvegarde des métriques
metrics_file = PROCESSED_DIR / "advanced_preprocessing_metrics.json"
with open(metrics_file, 'w', encoding='utf-8') as f:
    json.dump(quality_metrics, f, indent=2, ensure_ascii=False, default=str)
print(f"   Métriques sauvegardées: {metrics_file}")



   Métriques sauvegardées: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\processed\advanced_preprocessing_metrics.json


In [31]:

# Export CSV léger pour analyse externe
csv_file = PROCESSED_DIR / "articles_preprocessed_summary.csv"
df_export = df_clean_dedup[[
    'title', 'source', 'published', 'language', 'quality_score_advanced',
    'entities_total', 'sentence_count', 'readability_score', 'text_complexity'
]].copy()
df_export.to_csv(csv_file, index=False, encoding='utf-8')
print(f"   Export CSV résumé: {csv_file}")


   Export CSV résumé: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\processed\articles_preprocessed_summary.csv


In [32]:
# Export JSON du corpus de calibration (pour Phase 3)
calibration_json = PROCESSED_DIR / "calibration_corpus_300.json"
calibration_export = calibration_corpus[[
    'id', 'title', 'text_cleaned', 'source', 'published', 'language',
    'quality_score_advanced', 'entities_advanced'
]].to_dict('records')

with open(calibration_json, 'w', encoding='utf-8') as f:
    json.dump(calibration_export, f, ensure_ascii=False, indent=2, default=str)
print(f"   Corpus calibration JSON: {calibration_json}")


   Corpus calibration JSON: C:\Users\beedi.goua_square-ma\Desktop\Gheb\projet perso\InsightDetector\insight-detector\data\processed\calibration_corpus_300.json


In [33]:
print(f"RÉSULTATS FINAUX:")
print(f"   Source: {quality_metrics['source_file']}")
print(f"   Mode: {'Enrichissement complémentaire' if ENRICHED_MODE else 'Pipeline complet'}")
print(f"   Articles traités: {quality_metrics['total_articles_input']}")
print(f"   Articles finaux: {quality_metrics['articles_after_deduplication']}")
print(f"   Corpus de calibration: {quality_metrics['calibration_corpus_size']}")
print(f"   Taux de déduplication: {quality_metrics['deduplication_rate']:.1%}")



RÉSULTATS FINAUX:
   Source: enriched_articles.json
   Mode: Enrichissement complémentaire
   Articles traités: 200
   Articles finaux: 128
   Corpus de calibration: 186
   Taux de déduplication: 36.0%


In [34]:
print(f"\nMÉTRIQUES DE QUALITÉ:")
print(f"   Score qualité moyen: {quality_metrics['avg_quality_score']:.3f}")
print(f"   Entités par article: {quality_metrics['entities_avg_per_article']:.1f}")
print(f"   Biais temporel détecté: {quality_metrics['temporal_bias_detected']:.2f}")
print(f"   Biais géographique: {quality_metrics['geographic_bias_detected']:.2f}")




MÉTRIQUES DE QUALITÉ:
   Score qualité moyen: 0.681
   Entités par article: 23.1
   Biais temporel détecté: 0.81
   Biais géographique: -0.57


In [35]:
print(f"\nFICHIERS GÉNÉRÉS:")
print(f"   1. {output_file.name} - Dataset principal avec preprocessing avancé")
print(f"   2. {calibration_file.name} - Corpus stratifié pour calibration")
print(f"   3. {metrics_file.name} - Métriques détaillées")
print(f"   4. {csv_file.name} - Export CSV pour analyse")
print(f"   5. {calibration_json.name} - Corpus JSON pour Phase 3")





FICHIERS GÉNÉRÉS:
   1. articles_preprocessed_advanced.pkl - Dataset principal avec preprocessing avancé
   2. calibration_corpus_stratified.pkl - Corpus stratifié pour calibration
   3. advanced_preprocessing_metrics.json - Métriques détaillées
   4. articles_preprocessed_summary.csv - Export CSV pour analyse
   5. calibration_corpus_300.json - Corpus JSON pour Phase 3


In [36]:
# Variables exportées pour les autres notebooks
print(f"\nVARIABLES DISPONIBLES POUR EXPORT:")
print(f"   - df_clean_dedup: DataFrame principal preprocessé")
print(f"   - calibration_corpus: Corpus de calibration")
print(f"   - quality_metrics: Métriques de qualité")
print(f"   - PROCESSED_DIR: Répertoire des données traitées")




VARIABLES DISPONIBLES POUR EXPORT:
   - df_clean_dedup: DataFrame principal preprocessé
   - calibration_corpus: Corpus de calibration
   - quality_metrics: Métriques de qualité
   - PROCESSED_DIR: Répertoire des données traitées


In [37]:
# Sauvegarde des variables principales pour les autres notebooks
import pickle
variables_export = {
    'df_clean_dedup': df_clean_dedup,
    'calibration_corpus': calibration_corpus,
    'quality_metrics': quality_metrics,
    'PROCESSED_DIR': PROCESSED_DIR
}
with open(PROCESSED_DIR / 'preprocessing_variables.pkl', 'wb') as f:
    pickle.dump(variables_export, f)
print(f"   Variables sauvegardées: preprocessing_variables.pkl")

   Variables sauvegardées: preprocessing_variables.pkl
